In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from datasets import load_dataset
from transformers import GPT2TokenizerFast, DataCollatorForLanguageModeling

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
filepath = "/kaggle/input/drake-lyrics/drake_data.csv"
dataset = load_dataset('csv', data_files = filepath)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e68ced4a22278e52/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['album', 'lyrics_title', 'lyrics_url', 'lyrics', 'track_views'],
        num_rows: 290
    })
})

In [4]:
dataset['train'][11]

{'album': 'Dark Lane Demo Tapes',
 'lyrics_title': 'Chicago Freestyle (Ft. Giveon) Lyrics',
 'lyrics_url': 'https://genius.com/Drake-chicago-freestyle-lyrics',
 'lyrics': '[Chorus: Giveon]\nTwo-thirty, baby, won\'t you meet me by The Bean?\nToo early, maybe later you can show\u2005me\u2005things\nYou know what\u2005it is whenever I visit\nWindy city,\u2005she blowin\' me kisses, no\nThirty degrees, way too cold, so hold me tight\nWill I see you at the show tonight?\nWill I see you at the show tonight?\n\n[Verse 1: Drake]\nAyy, truck to the plane to the truck\nTruck to the hotel lobby\nMe, I go through underground garages\nPresidential suite on deposit\nElevator up to the room\nShower up and then we hit the club\nTouch down, gotta see what\'s up\nArea code in my phone\nWhat numbers do I still have?\nWho do I know from the past?\nHit one, she say she got a man\nHit another one, it goes green\nMust\'ve changed phones on the team\n\'Member when you let me in between\nThat was 2017\nAll goo

In [5]:
# model_id = "gpt2-medium"
# tokenizer = GPT2TokenizerFast.from_pretrained(model_id, padding = True)
model_id="gpt2-medium"
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

In [6]:
MAX_LENGTH = 256
BATCH_SIZE = 6
tokenized_data = tokenizer(
    dataset['train'][0]['lyrics'],
    truncation = True,
    max_length = MAX_LENGTH,
    return_overflowing_tokens = True,
    return_length = True
)

In [7]:
tokenized_data

{'input_ids': [[58, 13414, 325, 60, 198, 11588, 616, 7666, 319, 4771, 198, 30374, 587, 257, 16840, 198, 37608, 1431, 18854, 2933, 11, 7599, 991, 2612, 1203, 198, 28541, 318, 691, 651, 43701, 6, 38427]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'length': [33], 'overflow_to_sample_mapping': [0]}

In [8]:
def preprocess(example):
    try:
        outputs = tokenizer(
            example["lyrics"],
            truncation=True,
            max_length=MAX_LENGTH,
            return_overflowing_tokens=True,
            return_length=True,
        )
        input_batch = []
        for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
            if length==MAX_LENGTH:
                input_batch.append(input_ids)
                valid_input_ids=input_ids
        if len(input_batch)!=0:
            for i in range(BATCH_SIZE-len(input_batch)):
                input_batch.append(valid_input_ids)
    except Exception as e:
#         print(e)
        input_batch=[]
#         return
    return {"input_ids": input_batch}

In [9]:
tokenized_dataset = dataset.map(preprocess, remove_columns = dataset['train'].column_names)
# tokenized_dataset = data.map(preprocess)

  0%|          | 0/290 [00:00<?, ?ex/s]

In [10]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 290
    })
})

In [11]:
# tokenized_dataset['train'][110]

In [12]:
for i in range(10):
    print(i, len(tokenized_dataset['train'][i]['input_ids']))

0 0
1 0
2 6
3 6
4 6
5 6
6 6
7 0
8 6
9 6


In [13]:
def filter_none(example):
    if len(example['input_ids']) >= 1:
        return example

In [14]:
final_data = tokenized_dataset.filter(filter_none)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
final_data

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 270
    })
})

In [16]:
max_len = 0
for i in range(10):
    print(i, len(final_data['train'][i]['input_ids']))
    if len(final_data['train'][i]['input_ids']) >= max_len:
        max_len = len(final_data['train'][i]['input_ids'])
print(max_len)

0 6
1 6
2 6
3 6
4 6
5 6
6 6
7 6
8 6
9 6
6


In [17]:
tokenizer.pad_token = tokenizer.eos_token
data_col = DataCollatorForLanguageModeling(tokenizer, mlm = False, return_tensors = "tf") # mlm = masked language modeling

In [18]:
tf_train_dataset = final_data["train"].to_tf_dataset(
    columns = ['input_ids', 'attention_mask', 'labels'],
    shuffle = True,
    batch_size = 1,
    collate_fn = data_col
)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [19]:
for i in tf_train_dataset.take(1):
    print(i)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 11547,   771, ...,   198, 34104,  2237],
        [  274,   290, 14659, ...,   338,  2877,   287],
        [18358, 18334,   198, ...,  3918,   326,   314],
        [  925,   510,   198, ...,   588,   289,  3028],
        [  925,   510,   198, ...,   588,   289,  3028],
        [  925,   510,   198, ...,   588,   289,  3028]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6), dtype=int64, numpy=array([[1, 1, 1, 1, 1, 1]])>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 11547,   771, ...,   198, 34104,  2237],
        [  274,   290, 14659, ...,   338,  2877,   287],
        [18358, 18334,   198, ...,  3918,   326,   314],
        [  925,   510,   198, ...,   588,   289,  3028],
        [  925,   510,   198, ...,   588,   289,  3028],
        [  925,   510,   198, ...,   588,   289,  3028]]])>}


In [20]:
def swap_position(example):
    return {
        'input_ids': example['input_ids'],
        'attention_mask': tf.ones([1, BATCH_SIZE, MAX_LENGTH]),
        'labels': example['labels']
    }

In [21]:
train_data = tf_train_dataset.map(swap_position)

In [22]:
for i in train_data.take(1):
    print(i)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58,  5317,   305, ...,   262,  3872,    11],
        [  314,   307, 22404, ...,  4908,    11,   314],
        [ 1101,  1103,  8246, ...,   707,    70,   357],
        [12804, 39843,   828, ...,   502,   534, 22803],
        [  198,  5492,   836, ...,  2504,   338,   616],
        [ 1468,   256,   278, ...,   705,   368,   198]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6, 256), dtype=float32, numpy=
array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]], dtype=float32)>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58,  5317,   305, ...,   262,  3872,    11],
        [  314,   307, 22404, ...,  4908,    11,   314],
        [ 1101,  1103,  8246, ...,   707,    70,   357],
        [12804, 39843,   828, ...

In [23]:
unbatched_train_data = train_data.unbatch()

In [24]:
for i in unbatched_train_data.take(1):
    print(i)

{'input_ids': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58,  5317,   305, ...,  3708,  7298, 19753],
       [  290,   467,   284, ...,   198,   198,    58],
       [ 8134,  3201,   352, ...,    58,  8134,  3201],
       [ 8134,  3201,   352, ...,    58,  8134,  3201],
       [ 8134,  3201,   352, ...,    58,  8134,  3201],
       [ 8134,  3201,   352, ...,    58,  8134,  3201]])>, 'attention_mask': <tf.Tensor: shape=(6, 256), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>, 'labels': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58,  5317,   305, ...,  3708,  7298, 19753],
       [  290,   467,   284, ...,   198,   198,    58],
       [ 8134,  3201,   352, ...,    58,  8134,  3201],
       [ 8134,  3201,   352, ...,    58,  8134,  3201],
   

In [25]:
from transformers import TFGPT2LMHeadModel
model = TFGPT2LMHeadModel.from_pretrained(model_id)
model.summary()

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 354823168 
 r)                                                              
                                                                 
Total params: 354,823,168
Trainable params: 354,823,168
Non-trainable params: 0
_________________________________________________________________


In [26]:
from transformers import create_optimizer
num_train_steps = len(unbatched_train_data)
optimizer, schedule = create_optimizer(
    init_lr = 5e-5,
    num_warmup_steps = 1_000,
    num_train_steps = num_train_steps
)

model.compile(
    optimizer = optimizer
)

In [27]:
train_history = model.fit(unbatched_train_data, epochs = 5)

Epoch 1/5
270/270 [==============================] - 266s 687ms/step - loss: 3.2656
Epoch 2/5
270/270 [==============================] - 185s 686ms/step - loss: 2.9780
Epoch 3/5
270/270 [==============================] - 185s 687ms/step - loss: 2.7483
Epoch 4/5
270/270 [==============================] - 185s 687ms/step - loss: 2.4636
Epoch 5/5
270/270 [==============================] - 185s 686ms/step - loss: 2.2010


In [28]:
model.save_weights('gpt2_medium.h5')

In [29]:
input_text="true love shouldn't be this complicated"

In [30]:
input_ids = tokenizer(input_text, return_tensors="tf")["input_ids"]

In [31]:
import time
init_time=time.time()
output_greedy = model.generate(input_ids,max_length=256,do_sample=False)
print(tokenizer.decode(output_greedy[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
I just wanna be with you forever, forever, forever
I just wanna be with you forever, forever, forever
I just wanna be with you forever, forever, forever

[Verse 2: Drake]
I just wanna be with you forever, forever, forever
I just wanna be with you forever, forever, forever
I just wanna be with you forever, forever, forever
I just wanna be with you forever, forever, forever

[Chorus: Drake]
I just wanna be with you forever, forever, forever
I just wanna be with you forever, forever, forever
I just wanna be with you forever, forever, forever

[Verse 3: Drake]
I just wanna be with you forever, forever, forever
I just wanna be with you forever, forever, forever
I just wanna be with you forever, forever, forever

[Chorus: Drake]
I just wanna be with you forever, forever, forever
I just wanna be with you forever, forever, forever
I just wanna be with you forever, forever, forever

[Verse 4: Drake]
I just wanna be with you forever, forever, forever
I
87.

In [32]:
init_time=time.time()
output_beam = model.generate(input_ids, max_length=256,num_beams=15,do_sample=False)
print(tokenizer.decode(output_beam[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated

[Verse 2: Drake]
I don't know how to talk to you
I don't know how to be there when you need me
I don't know how to hold you when you need me
I don't know how to hold you when you need me
I don't know how to hold you when you need me

[Chorus: Drake]
I don't know how to talk to you
I don't know how to be there when you need me
I don't know how to hold you when you need me
I don't know how to hold you when you need me

[Verse 3: Drake]
I don't know how to talk to you
I don't know how to be there when you need me
I don't know how to hold you when you need me
I don't know how to hold you when you need me

[Chorus: Drake]
I don't know how to talk to you
I don't know how to be there when you need me
I don't know how to hold you when you need me
I don't know how to hold you when you need me

[Verse 4
104.21249103546143


In [33]:
init_time=time.time()
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=1.0, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated"
It's a lot easier to take my shit than it is to get it off
Start thinking, "I'm too scared to stay down, my problems probably don't need explaining"
When I say problems probably don't need explaining, I'm scared to lie
I can't trust nobody on these days
Long as you're using it for the wrong reasons

[Chorus: Drake]
Emotions clouding my judgment, my trust has been betrayed
When I need someone else the picture is clouded
My closest friends at work cannot tell the difference between us
When I'm certain that someone I used to know will forget me
This show is molding my ambition, my vision
I need someone I know better than others
That's why I trust you less than others
Don't trust nobody on these days
Your enemies on these days
Long as you're using it for the wrong reasons

Show me this time: 10 PM, New York City, outside, club, public bathroom
I promise we'll PM each other and nothing ever happens

[Verse 2: Chris Brown]
It's confusing seeing the th

In [34]:
init_time=time.time()
output_topk = model.generate(input_ids, max_length=256, do_sample=True,top_k=50)
print(tokenizer.decode(output_topk[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated'
Cause we can get confused and I know it
We should just go for it, we could split up
Maybe we'll fall in love and be together, but

[Bridge]
And say that we're best together but we have to try

[Verse 2]
I'm all alone, in a dark corner
I miss the city, how it used to be here
Doing these things that excites me
I should be where she at, in a different mood
Or when I sleep, I think about going there
Maybe she just missed the mark, but she is right out here
And I miss the way people approach me
I hope she doesn't find out

[Chorus]
But I do know, you can count on me, yeah
I do know, you can count on me and you know
I know when I say it, it's true, I think, oh, wait, was I in the wrong song?
No no no, I didn't say that
I know, it's true, I think, oh, wait, was I on tour?
No no no, I didn't say that
You can count on me
87.30979537963867


In [35]:
init_time=time.time()
output_topk = model.generate(input_ids, max_length=256, do_sample=True,temperature=2.0,top_k=50)
print(tokenizer.decode(output_topk[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
Baby what the other side can expect for me
Grow the fence like the rest (yeah but fuck me so I keep growing) Growing up a pro (Fuck a liar 'bout these lil muggables) A nigga a legend but all these lies they run out quickly
What it taking right here, this the second amendment that got to disappear, what's going on
And they always tryna get on another story, cause he got to, that's a major problem I'm seeing
Where they gotta build so no police, they call us to shut our door or shut our city down or something like this? They know we out here, the trouble start nite (It's off in a van but we move too slow), I ain't triyy (3 am) A new motto niggas can't forget, let me bring ya shit into The Met where no papa be stoopin (He don't have a chance wit he life) That ass ain't a Rolls they made for a Bess) Let y'all know this the address and phone number you should call if your life calls a thousand miles achive

Love doesn't start the way everybody in the c